In [21]:
import pandas as pd
import numpy as np
import MICE
import iMICE

df_ds = pd.read_csv('titanic.csv')
df=df_ds.copy()
df = df.drop(df.columns[[10]],axis = 1)
df = df.dropna(axis=0)

df_1=df.copy()
df_1['AgeBinary'] = df['Age'].apply(lambda x: 1 if x >= 19 else 0)

X = df_1.iloc[:, [2,9]].values  
y = df_1.iloc[:, 11].values

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

#actual
actual=np.concatenate((y_train,y_test),axis=0)
r1=len(y_train)

#y_test
df_2n = pd.DataFrame({'Age':y_test[:]})
df_2n.at[:,'Age']=np.nan
y_test_n=df_2n.iloc[:,0].values

y_train_1=np.concatenate((y_train,y_test_n),axis=0)
X_train_1=np.concatenate((X_train,X_test),axis=0)

df_2 = pd.DataFrame({'Age':y_train_1[:],'Pclass':X_train_1[:,0],'Fare':X_train_1[:,1]})


# MICE

In [25]:
mice_result = MICE.MICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

print(mice_result.shape)
df_3 = pd.DataFrame(mice_result, columns = ['Age', 'Pclass', 'Fare'])
predict = df_3['Age']

predict = np.array(predict)

from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix  
print(accuracy_score(y_test, predict[r1:]))
print(f1_score(y_test, predict[r1:]))
print(precision_score(y_test, predict[r1:]))
print(recall_score(y_test, predict[r1:]))

#print(classification_report(y_test, predict[r1:]))
print(confusion_matrix(actual[r1:], predict[r1:])) 


(712, 3)
0.7762237762237763
0.873015873015873
0.8148148148148148
0.9401709401709402
[[  1  25]
 [  7 110]]


# iMICE

In [26]:
imice_result = iMICE.iMICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

df_3i = pd.DataFrame(imice_result, columns = ['Age Binary', 'Pclass', 'Fare'])
predicti = df_3i['Age Binary'].map(lambda x: int(x))

from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix  
print(accuracy_score(y_test, predicti[r1:]))
print(f1_score(y_test, predict[r1:]))
print(precision_score(y_test, predict[r1:]))
print(recall_score(y_test, predict[r1:]))
print(confusion_matrix(actual[r1:], predicti[r1:]))

0.8251748251748252
0.873015873015873
0.8148148148148148
0.9401709401709402
[[  2  24]
 [  1 116]]


# KNN

In [27]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X_train, y_train)
y_pred = trained_model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.7552447552447552
[[  7  19]
 [ 16 101]]


# Random Forest

In [28]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=20, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.7692307692307693
[[  5  21]
 [ 12 105]]
